In [21]:
import numpy as np
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from datetime import datetime, timedelta
from h3.unstable import vect
from h3 import h3_to_geo, geo_to_h3
from scipy.sparse import coo_matrix
# from sklearn.preprocessing import normalize
import cartopy.crs as ccrs

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from tools import from_pickle, to_pickle, pcolorhex

def h3_cell_area():
    
    h3_index_upper = h3_index_base + np.uint64(1e17)
    
    dict_h3_normalize = pd.Series(index=map_h3_to_mat.index,dtype=np.float32)

    for i1 in range(len(depth_layers)-1):
        
        mask_select = (map_h3_to_mat.index < (h3_index_upper - index_remove_depth[i1])) & (map_h3_to_mat.index >= (h3_index_upper - index_remove_depth[i1+1]))
        
        h3_orig = map_h3_to_mat[mask_select].index + index_remove_depth[i1]
        
        areas = np.array([dict_coastlines[h3_orig[i1]]['area'] for i1 in range(len(h3_orig))]) *1e6 #m2
        
        
        dict_h3_normalize[map_h3_to_mat[mask_select].index] = areas#*height
     
    #coastlines
    mask_select = map_h3_to_mat.index > h3_index_upper
    h3_orig = map_h3_to_mat[mask_select].index - np.uint64(1e17)
    lengths = np.array([dict_coastlines[h3_orig[i1]]['length_coast_D1.27'] for i1 in range(len(h3_orig))]) *1e3
    dict_h3_normalize[map_h3_to_mat[mask_select].index] = lengths

    return dict_h3_normalize    

def h3_cell_volume():
    
    h3_index_upper = h3_index_base + np.uint64(1e17)
    
    dict_h3_normalize = pd.Series(index=map_h3_to_mat.index,dtype=np.float32)

    for i1 in range(len(depth_layers)-1):
        
        mask_select = (map_h3_to_mat.index < (h3_index_upper - index_remove_depth[i1])) & (map_h3_to_mat.index >= (h3_index_upper - index_remove_depth[i1+1]))
        
        h3_orig = map_h3_to_mat[mask_select].index + index_remove_depth[i1]
        
        areas = np.array([dict_coastlines[h3_orig[i1]]['area'] for i1 in range(len(h3_orig))]) *1e6 #m2
        
        if i1 == len(depth_layers)-2: #in the lowest layer, calculate difference between local bathymetry and upper depth level (otherwise this results in np.inf)
            bath_ = df_bath[h3_orig].values
            height = bath_ - depth_layers[i1]
        else:
            height = depth_layers[i1+1] - depth_layers[i1]

        # dict_h3_area[h3_orig] = areas
        # dict_h3_volume[h3_orig] = areas*height
        
        dict_h3_normalize[map_h3_to_mat[mask_select].index] = areas*height
     
    #coastlines
    mask_select = map_h3_to_mat.index > h3_index_upper
    h3_orig = map_h3_to_mat[mask_select].index - np.uint64(1e17)
    lengths = np.array([dict_coastlines[h3_orig[i1]]['length_coast_D1.27'] for i1 in range(len(h3_orig))]) *1e3
    dict_h3_normalize[map_h3_to_mat[mask_select].index] = lengths

    return dict_h3_normalize

def h3_cell_location():
 
    h3_index_upper = h3_index_base + np.uint64(1e17)
    
    df_h3_location = pd.DataFrame(index=map_h3_to_mat.index,columns=('lon','lat'),dtype=np.float32)

    for i1 in range(len(depth_layers)-1):
        
        mask_select = (map_h3_to_mat.index < (h3_index_upper - index_remove_depth[i1])) & (map_h3_to_mat.index >= (h3_index_upper - index_remove_depth[i1+1]))
        
        h3_orig = map_h3_to_mat[mask_select].index + index_remove_depth[i1]
        
        lons_ = []
        lats_ = []
        for i2 in range(len(h3_orig)):
            (lat,lon) = h3_to_geo(hex(h3_orig[i2]))
            lats_.append(lat)
            lons_.append(lon)
            
        
        df_h3_location.loc[map_h3_to_mat[mask_select].index,'lon'] = np.array(lons_)
        df_h3_location.loc[map_h3_to_mat[mask_select].index,'lat'] = np.array(lats_)
        
     
    #coastlines
    mask_select = map_h3_to_mat.index > h3_index_upper
    h3_orig = map_h3_to_mat[mask_select].index - np.uint64(1e17)

    lons_ = []
    lats_ = []
    for i2 in range(len(h3_orig)):
        (lat,lon) = h3_to_geo(hex(h3_orig[i2]))
        lats_.append(lat)
        lons_.append(lon)
    df_h3_location.loc[map_h3_to_mat[mask_select].index,'lon'] = np.array(lons_)
    df_h3_location.loc[map_h3_to_mat[mask_select].index,'lat'] = np.array(lats_)
    
    return df_h3_location          


# data_mass_balance = from_pickle('../00_assimilation_files/mass_balance.pickle')
# data_res = from_pickle('00_assimilation_files/8_iter_55_members_Year_2019_StokesFalse_2022-05-21_04:04.pickle') # big run, aadjusted bounds, log positively fouled
data = from_pickle('dict_results_nsteps924_Year_2019_StokesFalse_2022-10-12_20:57:16.509994.pickle')

h3_grid_res = 3
coastline_res = '50m'
depth_layers = np.array([0,5,50,500,np.inf])
data_folder = '../'
h3_index_base = np.uint64(5e17)
index_add_coast = np.uint64(1e17)     #indices for coastal cells are the ocean surface cells + index_add_coast
h3_index_upper = h3_index_base + np.uint64(1e17)
index_remove_depth = np.array([int(0),int(1e17),int(2e17),int(3e17),int(4e17)])

df_bath = from_pickle(os.path.join(data_folder,'00_data_files/bathymetry_h%i.pickle' % h3_grid_res))
filename_dict_transition = os.path.join(data_folder,'00_transition_files/transitions_h3_50m_sinking_l_0.000100-0.000400-0.001600-0.006400-0.025600-0.102400_year_2019_month_01-02-03-04-05-06-07-08-09-10-11.pickle')
P = from_pickle(filename_dict_transition)
map_mat_to_h3 = P['map_mat_to_h3']
map_h3_to_mat = P['map_h3_to_mat']
dict_coastlines = from_pickle(os.path.join(data_folder,'00_data_files/coastal_properties_h3_res%i_%s_Dc.pickle' % (h3_grid_res,coastline_res)))
h3_index_coast = np.array([hex_ for hex_ in dict_coastlines.keys() if 'length_coast' in dict_coastlines[hex_].keys() ],dtype=np.uint64)
dict_h3_normalize = h3_cell_volume()
dict_h3_location = h3_cell_location()
dict_h3_area = h3_cell_area()

# n_sizes = data_mass_balance['mass_removal_coast'][420].shape[1]
n_sizes = data[2015]['mass'].shape[1]
l0 = 1.6384  #TODO: change back to 0.1024
k_arr = np.arange(0,n_sizes) #11: 0.4mm to 0.4m
l_arr = l0 / (2**k_arr)    




In [22]:
h3_orig = map_h3_to_mat.index.values.copy()
depths = np.zeros(len(h3_orig),dtype=int)

for i1 in range(len(depth_layers)-1):
        
        mask_select = (map_h3_to_mat.index < (h3_index_upper - index_remove_depth[i1])) & (map_h3_to_mat.index >= (h3_index_upper - index_remove_depth[i1+1]))
        
        h3_orig[mask_select] = map_h3_to_mat[mask_select].index + index_remove_depth[i1]
        depths[mask_select] = (0 - i1)
        
mask_select = map_h3_to_mat.index > h3_index_upper
h3_orig[mask_select] = map_h3_to_mat[mask_select].index - np.uint64(1e17)
depths[mask_select] = 1

hexes = np.array([hex(h3_orig[i].astype(int)) for i in range(len(h3_orig)) ])    

In [26]:

data

{2015: {'mass': array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [2.02922999e+05, 9.41822292e+05, 1.65417354e+06, ...,
          1.68495314e+03, 9.91086665e+02, 6.02122876e+02],
         [7.48625863e+04, 3.49381354e+05, 6.19104232e+05, ...,
          1.69747611e+03, 1.12513980e+03, 7.59642461e+02],
         [3.30301974e+05, 1.54287583e+06, 2.75429965e+06, ...,
          9.49707841e+03, 5.75056856e+03, 3.45418651e+03]]),
  'num': array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00

In [63]:
types = ['mass','num','num_f']
units = ['total','concentration']

for year_ in data.keys():
    
    # type_ = types[0]
    # unit_ = units[1]
    
    for type_ in types:
        for unit_ in units:
            
            data_out = dict_h3_location.copy()
            data_out['h3_hex_ID'] = hexes
            data_out['depth_ID'] = depths

            str_output = '_'.join([str(year_),type_,unit_]) + '.csv'

            for i1,l_ in enumerate(l_arr):
                header = '_'.join([type_,unit_,'l=%2.2f[mm]'%(l_*1000)])
                if unit_ == 'total':
                    data_out[header] = data[year_][type_][:,i1]
                elif unit_ == 'concentration':
                    data_out[header] = data[year_][type_][:,i1] / dict_h3_normalize.values

            header = '_'.join([type_,unit_,'l=total'])
            if unit_ == 'total':
                data_out[header] = data[year_][type_].sum(axis=1)
            elif unit_ == 'concentration':
                data_out[header] = data[year_][type_].sum(axis=1) / dict_h3_normalize.values                       
            
            data_out.to_csv(str_output)



/tmp/ipykernel_253350/741306549.py:23: RuntimeWarning: divide by zero encountered in true_divide
  data_out[header] = data[year_][type_][:,i1] / dict_h3_normalize.values
/tmp/ipykernel_253350/741306549.py:29: RuntimeWarning: divide by zero encountered in true_divide
  data_out[header] = data[year_][type_].sum(axis=1) / dict_h3_normalize.values


In [61]:
data_out

,lon,lat,h3_hex_ID,depth_ID,mass_concentration_l=1638.40[mm],mass_concentration_l=819.20[mm],mass_concentration_l=409.60[mm],mass_concentration_l=204.80[mm],mass_concentration_l=102.40[mm],mass_concentration_l=51.20[mm],mass_concentration_l=25.60[mm],mass_concentration_l=12.80[mm],mass_concentration_l=6.40[mm],mass_concentration_l=3.20[mm],mass_concentration_l=1.60[mm],mass_concentration_l=0.80[mm],mass_concentration_l=0.40[mm],mass_concentration_l=0.20[mm],mass_concentration_l=0.10[mm],mass_concentration_l=total
0,NaN,NaN,0x0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0x1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0x2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289972169660825599,31.757227,81.957489,0x830008fffffffff,-3,7.803633e-12,3.532576e-11,5.891182e-11,3.822673e-11,1.179068e-11,4.091683e-12,2.704592e-12,1.062199e-11,1.385874e-11,3.943612e-11,4.501570e-11,1.999197e-11,1.963887e-12,5.343680e-12,1.047591e-11,3.055629e-10
289972238380302335,25.863445,82.748108,0x830009fffffffff,-3,4.315438e-13,1.954927e-12,3.265533e-12,2.134944e-12,6.755888e-13,2.398632e-13,1.566235e-13,4.762916e-13,5.706443e-13,1.506391e-11,2.065390e-11,9.532832e-12,1.675508e-13,5.574329e-13,1.619690e-12,5.750128e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690792817652006911,145.711304,44.855713,0x832eaefffffffff,1,8.791062e-01,4.087334e+00,7.182660e+00,5.111059e+00,2.088359e+00,8.931421e-01,4.630161e-01,2.331187e-01,1.131078e-01,5.986251e-02,3.129030e-02,1.823914e-02,1.222902e-02,7.364901e-03,4.295699e-03,2.118418e+01
690381737742172159,166.698853,59.892059,0x831750fffffffff,1,4.281696e-01,1.986312e+00,3.480638e+00,2.442358e+00,9.538581e-01,3.916763e-01,1.974669e-01,9.811816e-02,4.593030e-02,2.039699e-02,1.037649e-02,5.677040e-03,2.971710e-03,1.783091e-03,1.067513e-03,1.006680e+01
690505432800296959,30.387722,45.588078,0x831e58fffffffff,1,7.378920e-01,3.424762e+00,6.015096e+00,4.251543e+00,1.675821e+00,7.012836e-01,3.637832e-01,1.832874e-01,8.684840e-02,3.989575e-02,2.130443e-02,1.177140e-02,6.127020e-03,3.603903e-03,2.189508e-03,1.752521e+01
690062054736396287,91.265167,79.440659,0x830524fffffffff,1,2.386767e-01,1.113900e+00,1.973843e+00,1.451244e+00,6.373485e-01,3.091098e-01,1.779703e-01,9.995286e-02,5.435592e-02,2.949272e-02,1.643004e-02,1.001911e-02,5.414788e-03,3.589626e-03,2.424184e-03,6.123771e+00


In [58]:
data[year_][type_].sum(axis=1)

(120732,)

In [62]:
data_out.to_csv('test.csv')